In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sentence_transformers import CrossEncoder


# model = CrossEncoder("./huggingface_checkpoint_pointwise_crossencoder", max_length=512) 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Tymoteusz/distilbert-base-uncased-kaggle-readability")

model = AutoModelForSequenceClassification.from_pretrained("Tymoteusz/distilbert-base-uncased-kaggle-readability")

In [16]:
res = model(torch.tensor([tokenizer.encode("This is a test asdf asdf asdf asdf", add_special_tokens=True)]))
predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]
predicted_labels = [model.config.id2label[id.item()] for id in predicted_class_ids]
predicted_labels

['LABEL_0']

In [2]:

# print os cuda_home
import os
print(os.environ['CUDA_HOME'])

/home/jonas/miniconda3/envs/transformers


In [3]:
# print cuda home
print(torch.cuda.is_available())


False


In [4]:
import xml.etree.ElementTree as ET
import pandas as pd

def load_topics(path):
    with open(path) as f:
        root = ET.fromstring(f.read())
    topic_dict = {}
    for topic in root.findall("topic"):
        topic_id = topic.findtext("id")
        topic_query = topic.findtext("query")
        if topic_id and topic_query:
            topic_dict[topic_id] = topic_query.strip().lower()
    topics = pd.DataFrame(topic_dict.items(), columns=["qid", "query"]) 
    topics["query"] = topics["query"].str.replace(r'\W+', ' ', regex=True)
    return topics

In [10]:
all_passages = pd.read_csv("data/Webdoc/txt.tsv", sep="\t")


In [6]:
topics = load_topics("data/topics/topics.txt")

In [11]:
# generate list of querie, passage pairs. Each query will be paired with each passage
pairs = []
for index, row in topics.iterrows():
    if index > 0:
        break
    for index2, row2 in all_passages.iterrows():
        pairs.append((row["query"], row2["text"]))

In [12]:
len(pairs)

6692

In [14]:
# move to GPU

result = model.predict(pairs, show_progress_bar=True, batch_size=32)

Batches:   0%|          | 0/210 [00:00<?, ?it/s]

In [9]:
result

array([0.48864564, 0.5322153 , 0.4919514 , 0.4919514 , 0.49434176,
       0.5206046 , 0.5324732 , 0.529872  , 0.53022796, 0.53022796,
       0.5323814 , 0.5323214 , 0.5314924 , 0.5314924 , 0.528139  ,
       0.532533  , 0.52124846, 0.4583546 , 0.45850483, 0.45831004,
       0.53253686, 0.53206015, 0.5305847 , 0.5305847 , 0.5325372 ,
       0.4988671 , 0.45629552, 0.4665585 , 0.46510193, 0.46515056,
       0.52865934, 0.53140956, 0.5298428 , 0.5298428 , 0.52377075,
       0.52464134, 0.47937423, 0.52813023, 0.52801085, 0.52801085,
       0.50798106, 0.5128198 , 0.5065715 , 0.5065715 , 0.45762488,
       0.46619248, 0.46018952, 0.48184034, 0.48278084, 0.48162046,
       0.48510206, 0.49877754, 0.48057094, 0.48057094, 0.51463455,
       0.50622946, 0.5198705 , 0.5016975 , 0.5018771 , 0.5018771 ,
       0.53232324, 0.5323286 , 0.5314715 , 0.5314715 , 0.5322843 ,
       0.5320083 , 0.53246784, 0.52093726, 0.52153194, 0.52153194,
       0.49671698, 0.48679492, 0.50634   , 0.50634   , 0.50488